In [12]:
import tensorflow as tf
from simple_cnn_to_mnist import CNN_MNIST_MODEL

In [16]:
def read_tf_iterator(path,batch_size,is_shuffle_batch):
    tensor_path = tf.train.string_input_producer([path])
    tfreader = tf.TFRecordReader()
    _,serialstring = tfreader.read(tensor_path)
    feature = {
        'raw':tf.FixedLenFeature([],tf.string),
        "label":tf.FixedLenFeature([],tf.int64),
    }
    feature_tensor = tf.parse_single_example(serialstring,feature)
    image_tensor = tf.decode_raw(feature_tensor["raw"],tf.float32)
    image_tensor = tf.reshape(image_tensor, [28, 28, 1])
    label_tensor = tf.one_hot(feature_tensor["label"],10,1,0)
    if is_shuffle_batch >0:
        return tf.train.shuffle_batch([image_tensor,label_tensor],batch_size,32,31,15)
    else:
        return tf.train.batch([image_tensor,label_tensor],batch_size,100)

In [17]:
tf.reset_default_graph()
input_x = tf.placeholder(tf.float32,(None,28,28,1))
input_y = tf.placeholder(tf.float32,(None,10))
cnn_model = CNN_MNIST_MODEL(input_x,input_y,0.001,0.98,50)
prediction = cnn_model.forward()
train_op,loss,learning_rate,global_step,accuracy = cnn_model.train()

In [18]:
EPOCH = 3000
train_batch_size = 1024

batch_train_image_tensor,batch_train_label_tensor = read_tf_iterator("tfrecords/train.tf",train_batch_size,True)
val_image_tensor,val_label_tensor = read_tf_iterator("tfrecords/val.tf",5000,False)
test_image_tensor,test_label_tensor = read_tf_iterator("tfrecords/test.tf",10000,False)
print(batch_train_image_tensor.shape)
print(batch_train_label_tensor.shape)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess,coord)
    val_images,val_labels = sess.run([val_image_tensor,val_label_tensor])
    for epoch in range(EPOCH):
        train_batch_images,train_batch_labels = sess.run([batch_train_image_tensor,batch_train_label_tensor])
        sess.run(train_op,feed_dict={input_x:train_batch_images,input_y:train_batch_labels})
        if (epoch+1) % 100 == 0:
            ls,lr,gs,acc = sess.run([loss,learning_rate,global_step,accuracy],feed_dict={input_x:val_images,input_y:val_labels})
            print("Val[{}/{}]: loss={}, learning_rate={}, accuracy = {}, global_step={}".format(epoch+1,EPOCH,ls,lr,acc,gs))
    test_images,test_labels = sess.run([test_image_tensor,test_label_tensor])
    ls,lr,gs,acc = sess.run([loss,learning_rate,global_step,accuracy],feed_dict={input_x:test_images,input_y:test_labels})
    print("Test[{}/{}]: loss={}, learning_rate={}, accuracy = {}, global_step={}".format(epoch+1,EPOCH,ls,lr,acc,gs))
    coord.request_stop()
    coord.join()

(1024, 28, 28, 1)
(1024, 10)
Val[100/3000]: loss=558.01123046875, learning_rate=0.0009604000370018184, accuracy = 0.9675999879837036, global_step=100
Val[200/3000]: loss=334.6209716796875, learning_rate=0.0009223681990988553, accuracy = 0.9818000197410583, global_step=200
Val[300/3000]: loss=268.77593994140625, learning_rate=0.0008858424844220281, accuracy = 0.9846000075340271, global_step=300
Val[400/3000]: loss=252.91165161132812, learning_rate=0.0008507631719112396, accuracy = 0.9855999946594238, global_step=400
Val[500/3000]: loss=221.6634521484375, learning_rate=0.0008170729852281511, accuracy = 0.9869999885559082, global_step=500
Val[600/3000]: loss=240.51824951171875, learning_rate=0.0007847169181331992, accuracy = 0.9869999885559082, global_step=600
Val[700/3000]: loss=209.59498596191406, learning_rate=0.0007536421762779355, accuracy = 0.9879999756813049, global_step=700
Val[800/3000]: loss=207.44708251953125, learning_rate=0.0007237979443743825, accuracy = 0.9883999824523926, 